![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# **RelationExtractionApproach**

In this notebook, we will examine the `RelationExtractionApproach` to to train an `RelationExtractionModel`.


**📖 Learning Objectives:**

1. Learn how to preprocess your data before training a Relation Extraction model.

2. Understand how to train a Relation Extraction model using `RelationExtractionApproach`.

3. Understand how to resume Relation Extraction model training.




**🔗 Helpful Links:**

For extended examples of usage, see the [Spark NLP Workshop](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/03.0.Clinical_Relation_Extraction.ipynb)

Python Documentation: [RelationExtractionApproach](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/re/relation_extraction/index.html#sparknlp_jsl.annotator.re.relation_extraction.RelationExtractionApproach.name)

Scala Documentation: [RelationExtractionApproach](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/re/RelationExtractionApproach.html) <br/>



## **📜 Background**


`RelationExtractionApproach` is used for training a TensorFlow model for relation extraction. <br/>

To train a custom relation extraction model, you need to first creat a Tensorflow graph using either the `TfGraphBuilder` annotator or the `tf_graph` module. Then, set the path to the Tensorflow graph using the method `.setModelFile("path/to/tensorflow_graph.pb")`.

If the parameter `relationDirectionCol` is set, the model will be trained using the direction information (see the parameter decription for details). Otherwise, the model won’t have direction between the relation of the entities.

After training a model (using the .fit() method), the resulting object is of class `RelationExtractionModel`.








## **🎬 Colab Setup**

In [ ]:
!pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical

spark = nlp.start()

In [7]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

## **🖨️ Input/Output Annotation Types**

- Input: `WORD_EMBEDDINGS`, `POS`, `CHUNK`, `DEPENDENCY`

- Output: `CATEGORY`

## **📂 Training Data**

Downloading sample data:

In [8]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_clinical_rel_dataset.csv

In [9]:
data = spark.read.option("header","true").format("csv").load("i2b2_clinical_rel_dataset.csv")

data = data.select( 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel','dataset')

data.show(10)

# you only need these columns>> 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel'
# ('dataset' column is optional)

+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
|            sentence|firstCharEnt1|firstCharEnt2|lastCharEnt1|lastCharEnt2|              chunk1|              chunk2|   label1|   label2|  rel|dataset|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
|VITAL SIGNS - Tem...|           49|           75|          64|          84|    respiratory rate|          saturation|     test|     test|    O|   test|
|No lotions , crea...|            3|           34|           9|          42|             lotions|           incisions|treatment|  problem|TrNAP|   test|
|Because of expect...|           11|           58|          54|          68|expected long ter...|         a picc line|treatment|treatment|    O|  train|
|She states this l...|           16|           82|          31|          92|    li

In [10]:
data.groupby('dataset').count().show()

+-------+-----+
|dataset|count|
+-------+-----+
|  train|  350|
|   test|  650|
+-------+-----+



In [11]:
data.groupby('rel').count().show()

+-----+-----+
|  rel|count|
+-----+-----+
| TrIP|   14|
| TrAP|  164|
| TeCP|   26|
|    O|  414|
|TrNAP|   14|
| TrCP|   28|
|  PIP|  153|
| TrWP|   11|
| TeRP|  176|
+-----+-----+



In [12]:
#from sparknlp_jsl.training import REDatasetHelper

# map entity columns to dataset columns
column_map = {
    "begin1": "firstCharEnt1",
    "end1": "lastCharEnt1",
    "begin2": "firstCharEnt2",
    "end2": "lastCharEnt2",
    "chunk1": "chunk1",
    "chunk2": "chunk2",
    "label1": "label1",
    "label2": "label2"
}

# apply preprocess function to dataframe
data = medical.REDatasetHelper(data).create_annotation_column(
    column_map,
    ner_column_name="train_ner_chunks" # optional, default train_ner_chunks
)

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

In [13]:
data.show(10)

+------------+---------+--------------------+------------+-------------+--------------------+-----+--------------------+--------------------+-------+-------------+---------+
|lastCharEnt1|   label1|    train_ner_chunks|lastCharEnt2|firstCharEnt2|              chunk1|  rel|            sentence|              chunk2|dataset|firstCharEnt1|   label2|
+------------+---------+--------------------+------------+-------------+--------------------+-----+--------------------+--------------------+-------+-------------+---------+
|          64|     test|[{chunk, 49, 64, ...|          84|           75|    respiratory rate|    O|VITAL SIGNS - Tem...|          saturation|   test|           49|     test|
|           9|treatment|[{chunk, 3, 9, lo...|          42|           34|             lotions|TrNAP|No lotions , crea...|           incisions|   test|            3|  problem|
|          54|treatment|[{chunk, 11, 54, ...|          68|           58|expected long ter...|    O|Because of expect...|         a

## **✅ Graph Creation**

We will use `TFGraphBuilder` annotator which can be used to create graphs automatically in the model training pipeline.

`TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the approach.

You can also create a custom graph by using `tf_graph` module in Spark NLP for Healthcare.

In [ ]:
!pip install -q tensorflow==2.11.0 tensorflow-addons

In [ ]:
'''
# custom graph

medical.tf_graph.print_model_params("relation_extraction")

medical.tf_graph.build("relation_extraction",
             build_params={"input_dim": 10000,
                          "output_dim": 10,
                          'batch_norm':1,
                          "hidden_layers": [300, 200],
                          "hidden_act": "relu",
                          'hidden_act_l2':1},
                          'model_location'=".",
                          'model_filename'="re_in6000D_out10.pb")
'''

'''
# ready to use graph
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/RE_in1200D_out20.pb
'''

In [15]:
graph_folder= "./tf_graphs"

In [16]:
re_graph_builder = medical.TFGraphBuilder()\
    .setModelName("relation_extraction")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"]) \
    .setLabelColumn("rel")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("re_graph.pb")\
    .setHiddenLayers([300, 200])\
    .setHiddenAct("relu")\
    .setHiddenActL2(True)\
    .setHiddenWeightsL2(False)\
    .setBatchNorm(False)

## **🔎 Parameters**


- `FromEntity`: (begin_col: str, end_col: str, label_col: str) Sets from entity

>`begin_col` Column that has a reference of where the chunk begins </br>

> `end_col`: Column that has a reference of where the chunk ends

> `label_col`: Column that has a reference what are the type of chunk

- `ToEntity`: (begin_col: str, end_col: str, label_col: str) Sets to entity

> `begin_col` Column that has a reference of where the chunk begins </br>

> `end_col`: Column that has a reference of where the chunk ends

> `label_col`: Column that has a reference what are the type of chunk

- `CustomLabels`: (labels: dict[str, str]) Sets custom relation labels

> `labels`: Dictionary which maps old to new labels

- `RelationDirectionCol`: (col: str) Relation direction column (possible values are: "none", "left" or "right"). If this parameter is not set, the model will not have direction between the relation of the entities

> `col` Column contains the relation direction values

- `PretrainedModelPath` (value: str) Path to an already trained model saved to disk, which is used as a starting point for training the new model

- `ОverrideExistingLabels` (bool) Whether to override already learned labels when using a pretrained model to initialize the new model. Default is ‘true’

- `batchSize`: (Int) Size for each batch in the optimization process

- `EpochsNumber` (Int) Maximum number of epochs to train

- `Dropout`: (Float) Dropout at the output of each layer

- `LearningRate`: (Float) Learning rate for the optimization process

- `OutputLogsPath`: (Str) Folder path to save training logs. If no path is specified, the logs won't be stored in disk. The path can be a local file path, a distributed file path (HDFS, DBFS), or a cloud storage (S3).

- `ModelFile`: (Str) The path to the Tensorflow graph

- `FixImbalance` (Float) Fix the imbalance in the training set by replicating examples of under represented categories

- `ValidationSplit` (Float) The proportion of training dataset to be used as validation set

- `OverrideExistingLabels` (Boolean) Controls whether to override already learned lebels when using a pretrained model to initialize the new model. A value of true will override existing labels

- `MultiClass` (Boolean) If multiClass is set, the model will return all the labels with corresponding scores. By default, multiClass is false.

- `ModelFile` (Str) Location of file of the model used for classification

- `MaxSyntacticDistance` (Int) Maximal syntactic distance, as threshold (Default: 0)














## **🦾 Model Training**

Now, we will define `RelationExtractionApproach` and generate a pipeline along with required annotators and TFGraph.

In [17]:
documenter = nlp.DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

words_embedder = nlp.WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

dependency_parser = nlp.DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

# set training params and upload model graph (see ../Healthcare/8.Generic_Classifier.ipynb)
reApproach = medical.RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(80)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setModelFile(f"{graph_folder}/re_graph.pb")\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setOutputLogsPath('/content')

finisher = nlp.Finisher()\
    .setInputCols(["relations"])\
    .setOutputCols(["relations_out"])\
    .setCleanAnnotations(False)\
    .setValueSplitSymbol(",")\
    .setAnnotationSplitSymbol(",")\
    .setOutputAsArray(False)

train_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    re_graph_builder,
    reApproach,
    finisher
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]


In [18]:
%%time
rel_model = train_pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}
relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 3.79 s, sys: 298 ms, total: 4.08 s
Wall time: 17.7 s


In [19]:
rel_model.stages

[DocumentAssembler_6af920082bd8,
 REGEX_TOKENIZER_09748e8634ee,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 TFGraphBuilderModel_a2779a0336f2,
 RelationExtractionModel_d6427987b089,
 Finisher_4cc624e0fa46]

In [20]:
rel_model.stages[-2]

RelationExtractionModel_d6427987b089

Saving the model

In [21]:
rel_model.stages[-2].write().overwrite().save('custom_RE_model')

### Evaluating The Performance of the Model

We will create a pipeline and use our trained RE model by calling with `.load` method. Then we will get predictions by transforming our test set on this pipeline.

In [23]:
customReModel = medical.RelationExtractionModel.load("custom_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations_pred")\
    .setMaxSyntacticDistance(0)

test_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    words_embedder,
    pos_tagger,
    dependency_parser,
    customReModel])

test_results = test_pipeline.fit(test_data).transform(test_data)

In [24]:
test_results.show(5)

+------------+---------+--------------------+------------+-------------+--------------------+-----+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|lastCharEnt1|   label1|    train_ner_chunks|lastCharEnt2|firstCharEnt2|              chunk1|  rel|            sentence|              chunk2|dataset|firstCharEnt1| label2|           sentences|              tokens|          embeddings|            pos_tags|        dependencies|      relations_pred|
+------------+---------+--------------------+------------+-------------+--------------------+-----+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          64|     test|[{chunk, 49, 64, ...|          84|           75|    respiratory rate|    O|VITAL S

When you check the `relations_pred` column, you can see that some of the relations between the chunk pairs were not detected this time.

Now we will get metrics using the ground truth (`rel`) and prediction (`relations_pred`) columns in the result dataframe.

In [25]:
pd_test_results = test_results.select('rel', 'relations_pred.result').toPandas()
pd_test_results.head()

,rel,result
0,O,[O]
1,TrNAP,[O]
2,PIP,[PIP]
3,TeRP,[TeRP]
4,TrAP,[TrAP]


We will explode the `result` column and fill null values as `O` label.

In [26]:
pd_test_results = pd_test_results.explode("result").fillna("O")
pd_test_results.result.value_counts()

O        343
PIP      106
TeRP      99
TrAP      96
TeCP       2
TrNAP      2
TrCP       1
TrIP       1
Name: result, dtype: int64

In [27]:
pd_test_results.rel.value_counts()

O        274
TeRP     116
TrAP     108
PIP       88
TrCP      20
TeCP      15
TrIP      11
TrNAP      9
TrWP       9
Name: rel, dtype: int64

In [28]:
from sklearn.metrics import classification_report
print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

              precision    recall  f1-score   support

           O       0.66      0.83      0.74       274
         PIP       0.59      0.72      0.65        88
        TeCP       0.50      0.07      0.12        15
        TeRP       0.76      0.65      0.70       116
        TrAP       0.64      0.56      0.60       108
        TrCP       0.00      0.00      0.00        20
        TrIP       1.00      0.09      0.17        11
       TrNAP       0.00      0.00      0.00         9
        TrWP       0.00      0.00      0.00         9

    accuracy                           0.66       650
   macro avg       0.46      0.32      0.33       650
weighted avg       0.63      0.66      0.63       650



### Training by Setting the Relation Direction

We have `setRelationDirectionCol` parameter that is used during training with a new separate column that specifies the relationship directions. The column should contain one of the following values:

 - `rightwards`: The first entity in the text is also the first argument of the relation (as well as the second entity in the text is the second argument). In other words, the relation arguments are ordered *left to right* in the text.
 - `leftwards`: The first entity in the text is the second argument of the relation (and the second entity in the text is the first argument).
 - `both`: Order doesn't matter (relation is symmetric).

Let's modify our example training dataset accordingly

In [29]:
data.columns

['lastCharEnt1',
 'label1',
 'train_ner_chunks',
 'lastCharEnt2',
 'firstCharEnt2',
 'chunk1',
 'rel',
 'sentence',
 'chunk2',
 'dataset',
 'firstCharEnt1',
 'label2']

In [30]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

@F.udf(T.StringType())
def encodeRelationDirection(rel, begin1, begin2):
    if rel != "O":
        if begin1 > begin2:
            return "leftwards"
        else:
            return "rightwards"
    else:
        return "both"



data = data.withColumn("rel_dir", encodeRelationDirection("rel", "firstCharEnt1", "lastCharEnt2"))

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

Checking the new training data with **rel_dir** column

In [31]:
train_data.select("chunk1","label1","label2","chunk2","rel","rel_dir").show(10)

+--------------------+---------+---------+--------------------+----+----------+
|              chunk1|   label1|   label2|              chunk2| rel|   rel_dir|
+--------------------+---------+---------+--------------------+----+----------+
|expected long ter...|treatment|treatment|         a picc line|   O|      both|
|    light-headedness|  problem|  problem|         diaphoresis| PIP|rightwards|
| po pain medications|treatment|  problem|            his pain|TrAP| leftwards|
|bilateral pleural...|  problem|  problem|increased work of...| PIP|rightwards|
|    her urine output|     test|  problem|           decreased|TeRP|rightwards|
|his psychiatric i...|  problem|  problem|his neurologic in...| PIP|rightwards|
|   white blood cells|     test|     test|     red blood cells|   O|      both|
|            chloride|     test|     test|                 bun|   O|      both|
|     further work-up|     test|  problem|his neurologic co...|TeCP|rightwards|
|         four liters|treatment|     tes

In [32]:
train_data\
    .selectExpr("concat(label1, \", \", label2) AS args", "rel", "rel_dir")\
    .groupBy("rel", "args", "rel_dir").count().where("count > 10").orderBy("rel").show(1000, truncate=False)

+----+--------------------+----------+-----+
|rel |args                |rel_dir   |count|
+----+--------------------+----------+-----+
|O   |treatment, treatment|both      |26   |
|O   |test, test          |both      |59   |
|O   |problem, problem    |both      |38   |
|PIP |problem, problem    |rightwards|47   |
|PIP |problem, problem    |leftwards |18   |
|TeRP|test, problem       |rightwards|57   |
|TrAP|treatment, problem  |rightwards|29   |
|TrAP|treatment, problem  |leftwards |27   |
+----+--------------------+----------+-----+



Generating a new `RelationExtractionApproach()` with `.setRelationDirectionCol("rel_dir")` and training a new model.

In [33]:
documenter = nlp.DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

words_embedder = nlp.WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

dependency_parser = nlp.DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reApproach = medical.RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(80)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setModelFile(f"{graph_folder}/re_graph.pb")\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setOutputLogsPath('/rel_log')\
    .setRelationDirectionCol("rel_dir")

train_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    re_graph_builder,
    reApproach,
    finisher
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]


In [34]:
%%time
rel_model = train_pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}
relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 766 ms, sys: 60.2 ms, total: 827 ms
Wall time: 6.89 s


In [35]:
rel_model.stages

[DocumentAssembler_b9ce0bf136eb,
 REGEX_TOKENIZER_d6c09339511d,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 TFGraphBuilderModel_f3f1c4b0c3bc,
 RelationExtractionModel_8664e0f7a637,
 Finisher_4cc624e0fa46]

Saving the model

In [36]:
rel_model.stages[-2].write().overwrite().save('rel_dir_RE_model')

**Evaluating The Performance of the Model Trained with Directions**

In [37]:
customReModelDir = medical.RelationExtractionModel.load("rel_dir_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations_pred")\
    .setMaxSyntacticDistance(0)

test_pipeline_dir = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    words_embedder,
    pos_tagger,
    dependency_parser,
    customReModelDir])

test_results_dir = test_pipeline_dir.fit(test_data).transform(test_data)

Now we will get metrics using the ground truth (`rel`) and prediction (`relations_pred`) columns in the result dataframe.

In [38]:
pd_test_results_dir = test_results_dir.select('rel', 'relations_pred.result').toPandas()
pd_test_results_dir.head()

,rel,result
0,O,[O]
1,TrNAP,[O]
2,PIP,[PIP]
3,TeRP,[TeRP]
4,TrAP,[TrAP]


We will explode the `result` column and fill null values as `O` label and get metrics.

In [39]:
from sklearn.metrics import classification_report

pd_test_results_dir = pd_test_results_dir.explode("result").fillna("O")

print(classification_report(pd_test_results_dir["rel"], pd_test_results_dir["result"]))

              precision    recall  f1-score   support

           O       0.67      0.87      0.76       274
         PIP       0.59      0.64      0.61        88
        TeCP       0.00      0.00      0.00        15
        TeRP       0.85      0.61      0.71       116
        TrAP       0.70      0.68      0.69       108
        TrCP       0.00      0.00      0.00        20
        TrIP       0.00      0.00      0.00        11
       TrNAP       0.14      0.11      0.12         9
        TrWP       0.00      0.00      0.00         9

    accuracy                           0.68       650
   macro avg       0.33      0.32      0.32       650
weighted avg       0.63      0.68      0.64       650



As you can see, this model is slightly better than the previous one.

### Resume RelationExtractionApproach Model Training

Steps:
- Train a new model for a few epochs.
- Load the same model and train for more epochs on the same taxnonomy, and evaluate.
- Train a model already trained on a different dataset.

**Split the test data into two parts:**
- We keep the first part separate and use it for training the model further, as it will be totally unseen data from the same taxonomy.

- The second part will be used to testing and evaluating

In [40]:
(test_data_1, test_data_2) = test_data.randomSplit([0.5, 0.5], seed = 100)

#### Train a new model, pause, and resume training on the same dataset.

**Create a Graph**

In [41]:
#from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder= "./tf_graphs"

re_graph_builder = medical.TFGraphBuilder()\
    .setModelName("relation_extraction")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"]) \
    .setLabelColumn("rel")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("re_graph.pb")\
    .setHiddenLayers([300, 200])\
    .setHiddenAct("relu")\
    .setHiddenActL2(True)\
    .setHiddenWeightsL2(False)\
    .setBatchNorm(False)

**Train for 100 epochs**

In [42]:
reApproach = medical.RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(100)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setModelFile(f"{graph_folder}/re_graph.pb")\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setOutputLogsPath('/content')\
    .setRelationDirectionCol("rel_dir")\
    .setMaxSyntacticDistance(10)

finisher = nlp.Finisher()\
    .setInputCols(["relations"])\
    .setOutputCols(["relations_out"])\
    .setCleanAnnotations(False)\
    .setValueSplitSymbol(",")\
    .setAnnotationSplitSymbol(",")\
    .setOutputAsArray(False)

train_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    re_graph_builder,
    reApproach,
    finisher
])

In [43]:
%%time
rel_model = train_pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}
relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 866 ms, sys: 35.8 ms, total: 902 ms
Wall time: 7.5 s


In [44]:
result_test = rel_model.transform(test_data_2)

**Evaluate**

In [45]:
result_test_df = result_test.select('rel', 'relations.result').toPandas()
result_test_df.sample(5)

,rel,result
69,TeRP,[O]
191,PIP,[PIP]
242,O,[O]
291,TrAP,[TrAP]
70,TeRP,[O]


In [46]:
from sklearn.metrics import classification_report

pd_test_results = result_test_df.explode("result").fillna("O")

print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

              precision    recall  f1-score   support

           O       0.58      0.93      0.71       136
         PIP       0.60      0.55      0.58        38
        TeCP       1.00      0.25      0.40         4
        TeRP       0.77      0.16      0.27        61
        TrAP       0.67      0.65      0.66        48
        TrCP       0.00      0.00      0.00        12
        TrIP       0.00      0.00      0.00         7
       TrNAP       0.00      0.00      0.00         5
        TrWP       0.00      0.00      0.00         3

    accuracy                           0.60       314
   macro avg       0.40      0.28      0.29       314
weighted avg       0.59      0.60      0.54       314



Save to disk

In [48]:
rel_model.stages[-2].write().overwrite().save('RE_model_100e')

#### Train using the saved model on unseen dataset

We use unseen data from the same taxonomy

In [50]:
reApproach_finetune = medical.RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(30)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setRelationDirectionCol("rel_dir")\
    .setPretrainedModelPath("RE_model_100e")\
    .setОverrideExistingLabels(False)

finetune_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    re_graph_builder,
    reApproach_finetune,
    finisher
])

In [51]:
%%time
rel_model = finetune_pipeline.fit(test_data_1)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}
relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 863 ms, sys: 34 ms, total: 897 ms
Wall time: 7.8 s


In [52]:
result = rel_model.transform(test_data_2)

**Evaluate**

In [53]:
result_test_df = result.select('rel', 'relations.result').toPandas()
result_test_df.sample(5)

,rel,result
87,O,[O]
14,TeCP,[TeRP]
77,TeCP,[TeRP]
153,O,[O]
132,TrAP,[]


In [54]:
from sklearn.metrics import classification_report

pd_test_results = result_test_df.explode("result").fillna("O")

print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

              precision    recall  f1-score   support

           O       0.77      0.87      0.82       136
         PIP       0.77      0.71      0.74        38
        TeCP       0.00      0.00      0.00         4
        TeRP       0.83      0.79      0.81        61
        TrAP       0.63      0.75      0.69        48
        TrCP       0.25      0.08      0.12        12
        TrIP       0.67      0.29      0.40         7
       TrNAP       0.00      0.00      0.00         5
        TrWP       0.33      0.33      0.33         3

    accuracy                           0.74       314
   macro avg       0.47      0.42      0.43       314
weighted avg       0.71      0.74      0.72       314



Save to disk

In [55]:
rel_model.stages[-2].write().overwrite().save('RE_model_finetuned')

#### Take a model trained on a different dataset (pretrained model) and train on this dataset

In [56]:
clinical_re_Model = medical.RelationExtractionModel()\
    .pretrained("re_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")

clinical_re_Model.getClasses()

re_clinical download started this may take some time.
[OK!]


['TrWP', 'TrNAP', 'TrCP', 'PIP', 'TeCP', 'TeRP', 'TrIP', 'TrAP', 'O']

**Now train a model using this model as base**

In [57]:
reApproach_finetune = medical.RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(30)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setPretrainedModelPath("/root/cache_pretrained/re_clinical_en_2.5.5_2.4_1596928426753")\
    .setОverrideExistingLabels(False)

finetune_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    re_graph_builder,
    reApproach_finetune,
    finisher
])

In [58]:
%%time
rel_model = finetune_pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}
relation_extraction graph exported to ./tf_graphs/re_graph.pb
CPU times: user 779 ms, sys: 20 ms, total: 799 ms
Wall time: 5.91 s


In [59]:
result = rel_model.transform(test_data_2)

**Evaluate**

In [60]:
result_test_df = result.select('rel', 'relations.result').toPandas()
result_test_df.sample(5)

,rel,result
157,O,[O]
31,O,[O]
261,TrAP,[TrAP]
145,O,[PIP]
177,PIP,[TrAP]


In [61]:
from sklearn.metrics import classification_report

pd_test_results = result_test_df.explode("result").fillna("O")

print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

              precision    recall  f1-score   support

           O       0.59      0.67      0.63       136
         PIP       0.38      0.13      0.20        38
        TeCP       0.00      0.00      0.00         4
        TeRP       0.80      0.57      0.67        61
        TrAP       0.36      0.73      0.48        48
        TrCP       1.00      0.08      0.15        12
        TrIP       1.00      0.14      0.25         7
       TrNAP       0.00      0.00      0.00         5
        TrWP       0.00      0.00      0.00         3

    accuracy                           0.54       314
   macro avg       0.46      0.26      0.26       314
weighted avg       0.57      0.54      0.51       314



Save to disk

In [62]:
rel_model.stages[-2].write().overwrite().save('RE_pretrained_model_finetuned')